In [ ]:
using Revise
using Plots
plotly()

using QSimulator

# Transmon Ramsey Oscillations

Starting from a coherence between two energy eigenstates we expect oscilltions at the detuning between the energy levels.  We'll work with natural units of GHz and ns for numerical stability reasons. We'll start with a single 3 level transmon in the lab frame with a 5 GHz qubit frequency and -200 MHz anharmonicity.

In [ ]:
# we create a specific QSystem
q0 = FixedDuffingTransmon("q0", 5, -0.2, 3)
# we can ask for the Hamiltonian of an QSystem as a Matrix
hamiltonian(q0)

In [ ]:
# as CompositeQSystems is a tensor product structure of QSystem's and is what all the solvers are built around
cqs = CompositeQSystem([q0]);
add_hamiltonian!(cqs, hamiltonian(q0), "q0")
hamiltonian(cqs)

In [ ]:
# evolve an initial superpositions state for 1 ns
times = collect(linspace(0,1,201))
ψ0 = (1/sqrt(2)) * Complex128[1; 1; 0]
ψs = unitary_state(cqs, times, ψ0);

In [ ]:
# plot the projection on to the initial state and we expect to see 5 GHz oscillations
signal = Float64[real(ψ0'*ψ) for ψ in ψs]
expected = 0.5 + 0.5*cos.(2π*5 * (linspace(0,1,201)))
p = plot(times, signal, linewidth=2, label="simulated")
plot!(p, times, expected, label="ideal")
xlabel!(p, "Time (ns)")

# Rabi Oscillations

Driving Rabi oscillations in the lab frame is a good example of a parametric time dependent Hamiltonian. The drive electric field couples to the transmon dipole or $X$ operator.

## Constant Drive

In [ ]:
function qubit_drive(q::QSimulator.QSystem, drive::Function)
    function add_drive_ham!(ham, idxs, t)
        pulse = 2π * drive(t)
        drive_ham = real(pulse) * X(q) + imag(pulse) * Y(q)
        QSimulator.expand_add!(ham, drive_ham, idxs)
    end
end

qubit_freq = 5.0
q0 = FixedDuffingTransmon("q0", qubit_freq, -0.2, 3)
cqs = CompositeQSystem([q0]);
add_hamiltonian!(cqs, hamiltonian(q0), "q0")
add_hamiltonian!(cqs, qubit_drive(q0, t -> 0.02*cos(2π*qubit_freq * t)), q0);
ψ_init = Complex128[1; 0; 0]
times = collect(linspace(0,100,101))
ψs = unitary_state(cqs, times, ψ_init);

In [ ]:
p = plot(times, [abs2(s[1]) for s in ψs], label="Ground State Pop.")
plot!(p, times, [abs2(s[2]) for s in ψs], label="Excited State Pop.")
xlabel!(p, "Time (ns)")

## Variable Amplitude Gaussian Pulse

In [ ]:
# write a helper function that returns the drive Hamiltonian at a particular point in time
function gaussian(pulse_length, pulse_freq, t; cutoff=2.5)
    σ = pulse_length/2/cutoff
    pulse = exp(-0.5*((t-pulse_length/2)/σ)^2)
    pulse * cos(2π*pulse_freq * t)
end

function flat(pulse_freq,  t)
   cos(2π*pulse_freq * t)
end

function qubit_drive(qubit::QSimulator.QSystem, drive::Function)
    function add_drive_ham!(ham, idxs, t)
        pulse = 2π * drive(t)
        drive_ham = real(pulse) * X(qubit) + imag(pulse) * Y(qubit)
        QSimulator.expand_add!(ham, drive_ham, idxs)
    end
end

In [ ]:
ψ0 = Complex128[1; 0; 0]
states = []
amps = 0.05 * linspace(0,1, 51)
pulse_length = 50.0
for amp = amps
    # three level transmon in the lab frame
    q0 = FixedDuffingTransmon("q0", 5, -0.2, 3)
    cqs = CompositeQSystem([q0]);
    add_hamiltonian!(cqs, hamiltonian(q0), "q0")
#     add_hamiltonian!(cqs, qubit_drive(q0, t -> amp*gaussian(pulse_length, 5.0, t)), q0);
    add_hamiltonian!(cqs, qubit_drive(q0, t -> amp*flat(5.0, t)), q0);
    ψs = unitary_state(cqs, [0, pulse_length], ψ0)
    push!(states, ψs[end])
end

In [ ]:
p = plot(amps*1e3, [abs2(s[1]) for s in states], label="Ground State Pop.")
plot!(p, amps*1e3, [abs2(s[2]) for s in states], label="Excited State Pop.")
xlabel!(p, "Peak Nutation Strength (MHz)")